<a href="https://colab.research.google.com/github/JASHANBHULLAR4759/Assessment-4-Final-Examination---CSC101-Introduction-to-Programming/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import pyinputplus as pyip
from datetime import datetime

FILENAME = "budget.csv"


# ---------- 1. load or create ----------
def load_or_create_budget(filename=FILENAME):
    """
    Load budget.csv if it exists, otherwise create empty DataFrame
    with the required columns.
    """
    try:
        df = pd.read_csv(filename)
        # make sure required columns exist (in case old file)
        expected_cols = ["date", "description", "type", "amount", "category"]
        for col in expected_cols:
            if col not in df.columns:
                df[col] = ""
        df = df[expected_cols]
        print(f"Loaded existing {filename}")
    except FileNotFoundError:
        df = pd.DataFrame(columns=["date", "description", "type", "amount", "category"])
        print("No existing file found. Created empty budget DataFrame.")
    return df


# ---------- 2. menu ----------
def show_menu():
    """Show main menu and return user's choice (1-4)."""
    print("\n=== Personal Budget Manager ===")
    print("1. Add transaction")
    print("2. View all transactions")
    print("3. View current balance")
    print("4. Save and exit")

    choice = pyip.inputChoice(
        ["1", "2", "3", "4"],
        prompt="Please choose an option (1-4): "
    )
    return choice


# ---------- 3. date helper (today? yes/no) ----------
def get_transaction_date():
    """
    Ask user if they want to use today's date.
    If yes -> return today's YYYY-MM-DD.
    If no  -> ask for a date with validation.
    """
    use_today = pyip.inputChoice(
        ["yes", "no"],
        prompt="Use today's date? (yes/no): "
    )
    if use_today == "yes":
        return datetime.now().strftime("%Y-%m-%d")
    else:
        date_obj = pyip.inputDate(
            prompt="Enter date (YYYY-MM-DD): ",
            formats=["%Y-%m-%d"]
        )
        return date_obj.strftime("%Y-%m-%d")


# ---------- 4. add transaction ----------
def add_transaction(df):
    """Prompt user for transaction details (validated) and add to DataFrame."""
    # date
    date_str = get_transaction_date()

    # description
    description = pyip.inputStr("Enter description: ")

    # type: income / expense (numbered menu so user can enter 1 or 2)
    tran_type = pyip.inputMenu(
        ["income", "expense"],
        prompt="Select transaction type:\n",
        numbered=True
    )

    # amount
    amount = pyip.inputFloat("Enter amount: ", min=0.01)

    # category:
    # if it's INCOME -> don't ask categories, just set to "income"
    # if it's EXPENSE -> ask user to choose a category
    if tran_type == "income":
        category = "income"
    else:
        category = pyip.inputMenu(
            ["transport", "food", "shopping", "bills", "other"],
            prompt="Select expense category:\n",
            numbered=True
        )

    new_row = {
        "date": date_str,
        "description": description,
        "type": tran_type,
        "amount": amount,
        "category": category
    }

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    print("Transaction added.")
    return df


# ---------- 5. view all transactions ----------
def view_all_transactions(df):
    """Display all transactions or a message if none exist."""
    print("\n--- All Transactions ---")
    if df.empty:
        print("No transactions yet.")
    else:
        print(df.to_string(index=False))


# ---------- 6. calculate balance (pure logic) ----------
def calculate_balance(df):
    """
    Calculate balance = total income - total expense.
    This is pure logic so we can test it with assert().
    """
    if df.empty:
        return 0.0

    income_total = df[df["type"] == "income"]["amount"].sum()
    expense_total = df[df["type"] == "expense"]["amount"].sum()
    balance = income_total - expense_total
    return balance


# ---------- 6b. run a small assert test ----------
def run_balance_test():
    """Small assert-based test for calculate_balance()."""
    test_df = pd.DataFrame({
        "date": ["2025-11-12", "2025-11-12"],
        "description": ["salary", "lunch"],
        "type": ["income", "expense"],
        "amount": [1000.0, 50.0],
        "category": ["income", "food"]
    })
    expected = 950.0
    result = calculate_balance(test_df)
    assert result == expected, f"Expected {expected}, got {result}"
    print("Balance function test passed ✅")


# ---------- 7. view current balance (UI) ----------
def view_current_balance(df):
    """Display income, expenses and current balance nicely."""
    balance = calculate_balance(df)
    income_total = df[df["type"] == "income"]["amount"].sum()
    expense_total = df[df["type"] == "expense"]["amount"].sum()

    print("\n--- Current Balance ---")
    print(f"Total income:  ${income_total:.2f}")
    print(f"Total expense: ${expense_total:.2f}")
    print(f"Current Balance: ${balance:.2f}")

    if balance < 0:
        print("Warning: You are overspending.")


# ---------- 8. save and exit ----------
def save_and_exit(df, filename=FILENAME):
    """Save DataFrame to CSV (overwrite) and exit."""
    if not df.empty:
        df = df.sort_values(by="date")
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}. Goodbye!")
    return df


# ---------- 9. main loop ----------
def main():
    df = load_or_create_budget()

    # run the test once at start (assignment requirement)
    run_balance_test()

    while True:
        choice = show_menu()

        if choice == "1":
            df = add_transaction(df)
        elif choice == "2":
            view_all_transactions(df)
        elif choice == "3":
            view_current_balance(df)
        elif choice == "4":
            save_and_exit(df)
            break


# run the program
main()


No existing file found. Created empty budget DataFrame.
Balance function test passed ✅

=== Personal Budget Manager ===
1. Add transaction
2. View all transactions
3. View current balance
4. Save and exit
Please choose an option (1-4): 1
Use today's date? (yes/no): yes
Enter description: job 1
Select transaction type:
1. income
2. expense
1
Enter amount: 1000


/tmp/ipython-input-3132812056.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


Transaction added.

=== Personal Budget Manager ===
1. Add transaction
2. View all transactions
3. View current balance
4. Save and exit
Please choose an option (1-4): 3

--- Current Balance ---
Total income:  $1000.00
Total expense: $0.00
Current Balance: $1000.00

=== Personal Budget Manager ===
1. Add transaction
2. View all transactions
3. View current balance
4. Save and exit
Please choose an option (1-4): 1
Use today's date? (yes/no): yes
Enter description: car ent
Select transaction type:
1. income
2. expense
2
Enter amount: 160
Select expense category:
1. transport
2. food
3. shopping
4. bills
5. other
4
Transaction added.

=== Personal Budget Manager ===
1. Add transaction
2. View all transactions
3. View current balance
4. Save and exit
Please choose an option (1-4): 2

--- All Transactions ---
      date description    type  amount category
2025-11-12       job 1  income  1000.0   income
2025-11-12     car ent expense   160.0    bills

=== Personal Budget Manager ===
1. Add 